# Oscillation Models Evaluator

Compare multiple RheoJAX oscillation models on polystyrene mastercurve data.

**Data:** examples/data/pyRheo/polystyrene/master_curve_ps_oscillation_data.csv

In [1]:
# Google Colab compatibility - uncomment if running in Colab
# !pip install -q rheojax
# from google.colab import drive
# drive.mount('/content/drive')


## Setup and Imports

In [2]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
%matplotlib inline

import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fractional_maxwell_model import FractionalMaxwellModel
from rheojax.models.generalized_maxwell import GeneralizedMaxwell
from rheojax.pipeline.base import Pipeline
from rheojax.transforms.mastercurve import Mastercurve

jax, jnp = safe_import_jax()
verify_float64()
np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
warnings.filterwarnings('ignore', category=RuntimeWarning)

def r2_complex(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    ss_res = np.sum(np.abs(y_true - y_pred) ** 2)
    ss_tot = np.sum(np.abs(y_true - np.mean(y_true)) ** 2)
    return float(1 - ss_res / ss_tot)


INFO:2025-12-06 05:05:33,431:jax._src.xla_bridge:808: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file)


Loading rheojax version 0.4.0


/Users/b80985/Projects/rheojax/.venv/lib/python3.13/site-packages/piblin_jax/__init__.py:28: UserWarning: GPU acceleration is only available on Linux with CUDA 12+. Using JAX in CPU mode.
  from . import backend, data, dataio, transform


## Load oscillation dataset (polystyrene mastercurve)

In [3]:
DATA_DIR = Path.cwd().parent / 'data' / 'pyRheo' / 'polystyrene'
df = pd.read_csv(DATA_DIR / 'master_curve_ps_oscillation_data.csv')
omega = df['Angular Frequency'].to_numpy()
Gp = df['Storage Modulus'].to_numpy()
Gpp = df['Loss Modulus'].to_numpy()
G_star = Gp + 1j*Gpp


## Evaluate multiple models

In [4]:
models = {
    'GM-4': GeneralizedMaxwell(n_modes=4, modulus_type='tensile'),
    'GM-6': GeneralizedMaxwell(n_modes=6, modulus_type='tensile'),
    'FracMax': FractionalMaxwellModel(),
}
results = []
for name, model in models.items():
    try:
        model.fit(omega, G_star, test_mode='oscillation', use_log_residuals=True)
        pred = model.predict(omega)
        if pred.ndim == 2:
            pred = pred[:,0] + 1j*pred[:,1]
        r2 = r2_complex(G_star, pred)
        results.append({'model': name, 'r2': r2})
    except Exception as exc:
        results.append({'model': name, 'r2': np.nan, 'error': str(exc)})

pd.DataFrame(results)


Starting least squares optimization | {'method': 'trf', 'n_params': 9, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.397652s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=50 | final_cost=1.057424e+11 | time=1.398s | final_gradient_norm=67696131.21140718


Starting least squares optimization | {'method': 'trf', 'n_params': 9, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 0.323854s


...

Element minimization: early termination at n_modes=5 (R²=0.999301 < threshold=0.999499)


Starting least squares optimization | {'method': 'trf', 'n_params': 4, 'loss': 'linear', 'ftol': 1e-06, 'xtol': 1e-06, 'gtol': 1e-06}


Timer: optimization took 1.194709s


Convergence: reason=`ftol` termination condition is satisfied. | iterations=30 | final_cost=7.099389e+00 | time=1.195s | final_gradient_norm=0.0065040219009454515


,model,r2
0,GM-4,0.998121
1,GM-6,0.999666
2,FracMax,-43733.637158


## Plot best-fit overlay

In [5]:
best_candidates = [r for r in results if np.isfinite(r.get('r2', np.nan))]
if not best_candidates:
    print('No successful models')
else:
    best = max(best_candidates, key=lambda x: x['r2'])
    best_model = models[best['model']]
    best_pred = best_model.predict(omega)
    if hasattr(best_pred, 'ndim') and best_pred.ndim == 2:
        best_pred = best_pred[:,0] + 1j*best_pred[:,1]
    freq_hz = omega/(2*np.pi)

    fig, ax = plt.subplots(figsize=(9,6))
    ax.loglog(freq_hz, np.real(G_star), 'o', label="G' data", alpha=0.5)
    ax.loglog(freq_hz, np.imag(G_star), 's', label='G" data', alpha=0.5)
    ax.loglog(freq_hz, np.real(best_pred), '-', label=f"G' {best['model']}")
    ax.loglog(freq_hz, np.imag(best_pred), '--', label=f'G" {best["model"]}')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Modulus (Pa)')
    ax.set_title(f"Best model: {best['model']} (R^2={best['r2']:.3f})")
    ax.grid(True, which='both', ls='--', alpha=0.4)
    ax.legend()
    plt.show()


/var/folders/rq/lgpr8fvj349cr_wvkzv99md80000gp/T/ipykernel_19406/2479057968.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
